# Ranging Calculations in SpaceLink

This notebook demonstrates the ranging functions available in the SpaceLink library.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

from spacelink.core import ranging

# Set default plot style
plt.rcParams["figure.figsize"] = [12, 8]

## PN Sequence Range Ambiguity

The range ambiguity is the maximum unambiguous range that can be measured with a given PN sequence. It depends on the sequence length and chip rate. The CCSDS and DSN ranging sequences all share the same length of exactly 1,009,470 chips, so the chip rate is the main determiner of the ambiguity distance.

In [ ]:
chip_rates = np.logspace(5, 7, 100) * u.Hz  # From 100 kHz to 10 MHz
range_clock_rates = chip_rates / 2
ambiguities = ranging.pn_sequence_range_ambiguity(range_clock_rates)

plt.figure(figsize=(8, 5))
plt.loglog(chip_rates.to(u.MHz), ambiguities.to(u.km))
plt.xlabel("Chip Rate (MHz)")
plt.ylabel("Range Ambiguity (km)")
plt.title(f"Range Ambiguity vs. Chip Rate")
plt.grid(True)
plt.show()

## Chip SNR Calculation

The chip SNR determines ranging jitter and acquisition time. It depends on the chip rate and $P_R/N_0$.

In [ ]:
prn0_values = np.arange(30, 70, 0.1) * u.dBHz
range_clock_rate = 1.0 * u.MHz
snr_values = ranging.chip_snr(range_clock_rate, prn0_values)

plt.figure(figsize=(8, 5))
plt.plot(prn0_values, snr_values)
plt.xlabel("$P_R/N_0$ (dBHz)")
plt.ylabel("Chip SNR (dB)")
plt.title(f"Chip SNR vs. $P_R/N_0$ for Range Clock Frequency = {range_clock_rate}")
plt.grid(True)
plt.show()

## Uplink Power Fractions

When an uplink carrier is modulated with both ranging and command data signals, the power is distributed among the residual carrier, ranging sidebands, and data sidebands. Let's explore how these power fractions vary with modulation indices.

Note that the ranging and data power fractions shown here are the *usable* power fractions in those sidebands, so the three components do not in general sum to 1.

First is a plot of the power fractions versus ranging modulation index with the data modulation index held constant.

In [ ]:
# Calculate power fractions for different ranging modulation indices
ranging_mod_idx = np.linspace(0.0, 1.5, 100) * u.rad
data_mod_idx = 1 / math.sqrt(2) * u.rad
mod_type = ranging.CommandMod.SINE_SUBCARRIER

# Calculate power fractions
carrier_power_frac = ranging.uplink_carrier_to_total_power(
    ranging_mod_idx, data_mod_idx, mod_type
)
ranging_power = ranging.uplink_ranging_to_total_power(
    ranging_mod_idx, data_mod_idx, mod_type
)
data_power_frac = ranging.uplink_data_to_total_power(
    ranging_mod_idx, data_mod_idx, mod_type
)
total = carrier_power_frac + ranging_power + data_power_frac

plt.figure()
plt.plot(ranging_mod_idx, carrier_power_frac, label="Residual Carrier Power")
plt.plot(ranging_mod_idx, ranging_power, label="Usable Ranging Power")
plt.plot(ranging_mod_idx, data_power_frac, label="Usable Data Power")
plt.plot(ranging_mod_idx, total, label="Sum")
plt.ylim(0, 1)
plt.xlabel("Ranging Modulation Index (RMS radians)")
plt.ylabel("Power Fraction")
plt.title(
    f"Power Distribution vs. Ranging Modulation Index (Data RMS Mod Index = {data_mod_idx:.3f})"
)
plt.legend()
plt.grid(True)
plt.show()

Next are a set of filled contour plots showing how the power fractions change as a function of both data and ranging modulation indices.

In [ ]:
mod_type = ranging.CommandMod.SINE_SUBCARRIER
mod_idx_vals = np.linspace(0.1, 1.5, 20) * u.rad

mod_idx_ranging, mod_idx_data = np.meshgrid(mod_idx_vals, mod_idx_vals)
carrier_power = ranging.uplink_carrier_to_total_power(
    mod_idx_ranging, mod_idx_data, mod_type
)
ranging_power = ranging.uplink_ranging_to_total_power(
    mod_idx_ranging, mod_idx_data, mod_type
)
data_power = ranging.uplink_data_to_total_power(mod_idx_ranging, mod_idx_data, mod_type)
total_power = carrier_power + ranging_power + data_power

# Plot power distributions as 2x2 heatmap grid
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Carrier Power (top-left)
im1 = axes[0, 0].contourf(
    mod_idx_ranging,
    mod_idx_data,
    carrier_power,
    levels=np.linspace(0, 1, 21),
    cmap="viridis",
    vmin=0,
    vmax=1,
)
axes[0, 0].set_xlabel("Ranging Modulation Index (RMS rad)")
axes[0, 0].set_ylabel("Data Modulation Index (RMS rad)")
axes[0, 0].set_title("Residual Carrier Power Fraction")
cbar1 = fig.colorbar(im1, ax=axes[0, 0])
cbar1.set_ticks(np.linspace(0, 1, 6).tolist())

# Ranging Power (top-right)
im2 = axes[0, 1].contourf(
    mod_idx_ranging,
    mod_idx_data,
    ranging_power,
    levels=np.linspace(0, 1, 21),
    cmap="viridis",
    vmin=0,
    vmax=1,
)
axes[0, 1].set_xlabel("Ranging Modulation Index (RMS rad)")
axes[0, 1].set_ylabel("Data Modulation Index (RMS rad)")
axes[0, 1].set_title("Usable Ranging Power Fraction")
cbar2 = fig.colorbar(im2, ax=axes[0, 1])
cbar2.set_ticks(np.linspace(0, 1, 6).tolist())

# Data Power (bottom-left)
im3 = axes[1, 0].contourf(
    mod_idx_ranging,
    mod_idx_data,
    data_power,
    levels=np.linspace(0, 1, 21),
    cmap="viridis",
    vmin=0,
    vmax=1,
)
axes[1, 0].set_xlabel("Ranging Modulation Index (RMS rad)")
axes[1, 0].set_ylabel("Data Modulation Index (RMS rad)")
axes[1, 0].set_title("Usable Data Power Fraction")
cbar3 = fig.colorbar(im3, ax=axes[1, 0])
cbar3.set_ticks(np.linspace(0, 1, 6).tolist())

# Total Power (bottom-right)
im4 = axes[1, 1].contourf(
    mod_idx_ranging,
    mod_idx_data,
    total_power,
    levels=np.linspace(0, 1, 21),
    cmap="viridis",
    vmin=0,
    vmax=1,
)
axes[1, 1].set_xlabel("Ranging Modulation Index (RMS rad)")
axes[1, 1].set_ylabel("Data Modulation Index (RMS rad)")
axes[1, 1].set_title("Sum of Usable and Residual Carrier Powers")
cbar4 = fig.colorbar(im4, ax=axes[1, 1])
cbar4.set_ticks(np.linspace(0, 1, 6).tolist())

plt.tight_layout()
plt.show()